<a href="https://colab.research.google.com/github/akingno/colab_temp/blob/main/janus7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

In [2]:
!git clone https://github.com/deepseek-ai/Janus.git

fatal: destination path 'Janus' already exists and is not an empty directory.


In [3]:
ls

Janus/  sample_data/


In [4]:
cd Janus

/content/Janus


In [5]:
!pip install -e .

Obtaining file:///content/Janus
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for janus (pyproject.toml) ... done
  Created wheel for janus: filename=janus-1.0.0-0.editable-py3-none-any.whl size=15915 sha256=8ae21b811d445281fe86601ba57b06b1c8f8c31d550c73ff5bd46d1ad208c624
  Stored in directory: /tmp/pip-ephem-wheel-cache-rm6v83a7/wheels/04/ee/d6/76a460ef4080a263aa86cc3fdbb1c5bb29f559fbd8155d1c83
Successfully built janus
  Attempting uninstall: janus
    Found existing installation: janus 1.0.0
    Uninstalling janus-1.0.0:
      Successfully uninstalled janus-1.0.0


In [ ]:
import torch
from PIL import Image
from janus.models import MultiModalityCausalLM, VLChatProcessor
from janus.utils.io import load_pil_images
from datasets import load_dataset
import json
from google.colab import files  # 用于下载文件

# 加载模型和处理器
model_path = "deepseek-ai/Janus-Pro-7B"
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: MultiModalityCausalLM = MultiModalityCausalLM.from_pretrained(
    model_path, trust_remote_code=True,
)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

# 加载测试数据集
dataset = load_dataset("AdaptLLM/biomed-VQA-benchmark", "SLAKE", split="test")

results = []
for idx in range(10):  # 测试前10个样本
    sample = dataset[idx]

    try:
        # 处理输入数据
        question = sample['input']
        image = sample['image']  # 图像输入

        # 将图像转换为 PIL 格式
        pil_images = load_pil_images([image])

        # 构建对话结构
        conversation = [
            {
                "role": "<|User|>",
                "content": f"<image_placeholder>\n{question}",
                "images": pil_images,
            },
            {"role": "<|Assistant|>", "content": ""}
        ]

        # 准备输入数据
        prepare_inputs = vl_chat_processor(
            conversations=conversation, images=pil_images, force_batchify=True
        ).to(vl_gpt.device)

        # 获取图像嵌入
        inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

        # 生成回答
        outputs = vl_gpt.language_model.generate(
            inputs_embeds=inputs_embeds,
            attention_mask=prepare_inputs.attention_mask,
            pad_token_id=tokenizer.eos_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            max_new_tokens=150,
            do_sample=False,
            use_cache=True,
        )

        answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True).strip()

    except Exception as e:
        answer = f"Error: {str(e)}"

    # 收集结果
    results.append({
        "question": sample['input'],
        "generated_answer": answer,
        "image_id": sample.get('image_id', idx),  # 假设有image_id字段
    })

# 保存结果到 JSON 文件
output_file = "/content/generated_results.json"
with open(output_file, "w") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

# 下载保存的 JSON 文件
files.download(output_file)
print(f"测试完成，结果已保存并下载到 {output_file}")


Python version is above 3.10, patching the collections module.


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/image_processing_auto.py:594: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]